In [ ]:
import requests
import datetime
import pandas as pd
import numpy as np
from io import StringIO
import time

In [7]:
def parseData(date):
    date_str = date.strftime('%Y%m%d')

    data = requests.get('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date_str+'&type=ALLBUT0999', timeout=10)
    if data.text == '':
        return pd.DataFrame()

    index = 0
    證券代號index = 0
    for i in data.text.split('\n')[:1000]:
        if '證券代號' in i:
            # print(i)
            # print(index)
            證券代號index = index
        index = index + 1

    if 證券代號index == 0:

        return pd.DataFrame()
    df = pd.read_csv(StringIO(data.text.replace('=','')), header=證券代號index-1)
    # print(df)
    df = df.set_index(df['證券代號'].astype(str).str.replace(' ','')).drop(['證券代號'],axis=1)
    df = df.dropna(axis=1,how='all').dropna(axis=0,how='all')
    df.index.name = 'stock_id'
    df['date'] = pd.to_datetime(date)
    df = df.reset_index().set_index(['stock_id','date'])
    df = df.apply(lambda s: s.astype(str).str.replace(',',''))

    return df

def crawl_price(date):


    price_data = parseData(date)

    if len(price_data) != 0:
        print('成功')
        return price_data
    else:
        print('沒資料（可能是本日沒開盤/資料來源未更新/ip被擋）')
        return pd.DataFrame()
    time.sleep(4)

price = crawl_price(datetime.date(2023,8,25))
price

成功


,,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
stock_id,date,,,,,,,,,,,,,,,
0050,2023-08-25,元大台灣50,8003807,19366,1000262751,125.50,125.50,124.50,124.55,-,2.45,124.50,75,124.55,59,0.00
0051,2023-08-25,元大中型100,92241,261,6496090,71.40,71.40,70.05,70.05,-,1.55,70.15,3,70.90,53,0.00
0052,2023-08-25,富邦科技,490806,537,57142095,117.15,117.15,115.80,116.00,-,2.85,116.00,1,116.10,54,0.00
0053,2023-08-25,元大電子,4942,914,331732,67.20,67.55,67.00,67.55,-,1.4,67.15,1,67.50,22,0.00
0055,2023-08-25,元大MSCI金融,182368,268,4223878,23.23,23.23,23.08,23.08,-,0.15,23.08,4,23.11,3,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2023-08-25,新麗,107633,162,2115500,19.65,19.75,19.60,19.75,+,0.05,19.60,1,19.75,3,5.34
9945,2023-08-25,潤泰新,10643352,5030,367307337,33.10,35.25,33.10,34.50,+,1.2,34.45,47,34.50,55,25.00
9946,2023-08-25,三發地產,184777,114,2546935,13.85,13.90,13.65,13.75,X,0.0,13.75,14,13.80,2,31.98


In [ ]:
import datetime
import time
import csv
import os

if os.path.exists('./price.csv'):
    with open('./price.csv', 'r', newline='',encoding='latin-1') as csvfile:
        rows = csv.reader(csvfile)
        if len(list(rows))==0:
            fwrite = open('./price.csv', 'w')
            writer = csv.writer(fwrite)
            writer.writerow(['stock_id','date','證券名稱','成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價','漲跌(+/-)','漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比'])
            fwrite.close()
else:
    fwrite = open('./price.csv', 'w')
    writer = csv.writer(fwrite)
    writer.writerow(['stock_id','date','證券名稱','成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價','漲跌(+/-)','漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比'])
    fwrite.close()


start = '20230801'
end = '2023825'

datestart = datetime.datetime.strptime(start,'%Y%m%d')
dateend = datetime.datetime.strptime(end,'%Y%m%d')

#price_df = pd.read_csv('./price.csv',index_col=['stock_id','date'],parse_dates=['date'])
price_df = pd.read_csv('./price.csv',index_col=['stock_id','date'],parse_dates=['date'],encoding='latin-1')

while datestart<=dateend:
    print(datestart)
    if datestart not in pd.to_datetime(price_df.index.get_level_values('date')):

        price_df_new = crawl_price(datestart)

        price_df = pd.concat([price_df,price_df_new])

        price_df.to_csv('./price.csv')
    else:
        print('已存過日期')
    datestart=datestart+datetime.timedelta(days=1)



In [ ]:
price_df = pd.read_csv('./price.csv',index_col=['stock_id','date'],parse_dates=['date'])
price_df